<a href="https://colab.research.google.com/github/sanjeesi/Notes-Notebooks/blob/master/Data%20Science%20IITM/TDS/Module2%3A%20Get%20the%20Data/Web_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping with Excel
Import data from Web -> Set up **ETL** pipeline -> Refresh to get the latest data.  
> Keep the URL which has contains the table.  

1. Data Tab -> Query (New Query) -> From Other Sources -> From Web
2. Paste URL
3. Select the table which you want (**Extract**)
4. **Load** the data
5. Edit the query (**Transform**)
6. Close and load.


# BBC Weather location API with Python

Every set of (lat, long) is tagged to a location ID.  
Once we have the ID it's very easy to hit the database and pull the weather information for that particular location ID.  
If we scrape the location ID, we would be able to scrape the weather details of that city.  
> Get the API URL from the Network Inspector.

In [71]:
import os

import requests     # to get the webpage
import json         # to convert API to json format

from urllib.parse import urlencode
import numpy as np
import pandas as pd
import re           # regular expression operators

In [72]:
test_city = "Haridwar"
location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
    'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
    's': test_city,
    'stack': 'aws',
    'locale': 'en',
    'filter': 'international',
    'place-types': 'settlement,airport,district',
    'order': 'importance',
    'a': 'true',
    'format': 'json'
})
location_url

'https://locator-service.api.bbci.co.uk/locations?api_key=AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv&s=Haridwar&stack=aws&locale=en&filter=international&place-types=settlement%2Cairport%2Cdistrict&order=importance&a=true&format=json'

In [73]:
result = requests.get(location_url).json()
result

{'response': {'results': {'results': [{'container': 'India',
     'containerId': 1269750,
     'country': 'IN',
     'id': '1270351',
     'language': 'en',
     'latitude': 29.94791,
     'longitude': 78.16025,
     'name': 'Haridwar',
     'placeType': 'settlement',
     'timezone': 'Asia/Kolkata'}],
   'totalResults': 1}}}

In [74]:
# Print location id
result['response']['results']['results'][0]['id']

'1270351'

In [75]:
def getlocid(city):
  city = city.lower()     # to standardize format
  # Convert into an API call using URL encoding
  location_url = 'https://locator-service.api.bbci.co.uk/locations?' + urlencode({
    'api_key': 'AGbFAKx58hyjQScCXIYrxuEwJh2W2cmv',
    's': city,
    'stack': 'aws',
    'locale': 'en',
    'filter': 'international',
    'place-types': 'settlement,airport,district',
    'order': 'importance',
    'a': 'true',
    'format': 'json'
  })
  result = requests.get(location_url).json()
  locid = result['response']['results']['results'][0]['id']
  return locid

In [76]:
getlocid('Jamshedpur')

'1269300'

Now let's use this location ID to get the weather forecast for the next 14 days and save it as a csv file.  

*Web scraping might not be always legal. It is a good idea to check the terms of the website you plan to scrape before proceeding. Also, if your code requests a url from a server multiple times, it is a good practice to either cache your requests, or insert a timed delay between consecutive requests.*

In [77]:
from bs4 import BeautifulSoup   # to parse the webpage
from datetime import datetime

In [78]:
required_city = 'Jamshedpur'
url = 'https://www.bbc.com/weather/'+ getlocid(required_city)
response = requests.get(url)

In [79]:
soup = BeautifulSoup(response.content, 'html.parser')

In [80]:
# we want daily high, low temp and daily weather summary
daily_high_values = soup.find_all('span', attrs={'class': 'wr-day-temperature__high-value'})
daily_high_values

[<span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">35°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">95°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">37°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">98°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">38°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">101°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">39°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">103°</span></span></span>,
 <span class="wr-day-temperature__high-value"><span class="wr-value--temperature "><span c

In [81]:
daily_low_values = soup.find_all('span', attrs={'class': 'wr-day-temperature__low-value'})
daily_low_values

[<span class="wr-day-temperature__low-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">26°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">78°</span></span></span>,
 <span class="wr-day-temperature__low-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">24°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">75°</span></span></span>,
 <span class="wr-day-temperature__low-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">23°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">74°</span></span></span>,
 <span class="wr-day-temperature__low-value"><span class="wr-value--temperature "><span class="wr-value--temperature--c">25°</span><span class="wr-hide"> </span><span class="wr-value--temperature--f">76°</span></span></span>,
 <span class="wr-day-temperature__low-value"><span class="wr-value--temperature "><span class="w

In [82]:
daily_summary = soup.find('div', attrs={'class': 'wr-day-summary'})
daily_summary

<div class="wr-day-summary"><div class="gel-wrap"><span class="">Partly cloudy and light winds</span><span class="wr-hide">Thundery showers and light winds</span><span class="wr-hide">Sunny intervals and light winds</span><span class="wr-hide">Sunny intervals and light winds</span><span class="wr-hide">Sunny intervals and light winds</span><span class="wr-hide">Sunny intervals and light winds</span><span class="wr-hide">Sunny intervals and light winds</span><span class="wr-hide">Sunny intervals and light winds</span><span class="wr-hide">Light cloud and light winds</span><span class="wr-hide">Light cloud and light winds</span><span class="wr-hide">Sunny intervals and light winds</span><span class="wr-hide">Sunny intervals and a gentle breeze</span><span class="wr-hide">Sunny intervals and a gentle breeze</span><span class="wr-hide">Sunny intervals and a gentle breeze</span></div></div>

In [83]:
daily_summary.text

'Partly cloudy and light windsThundery showers and light windsSunny intervals and light windsSunny intervals and light windsSunny intervals and light windsSunny intervals and light windsSunny intervals and light windsSunny intervals and light windsLight cloud and light windsLight cloud and light windsSunny intervals and light windsSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breeze'

In [84]:
daily_high_values[0].text.strip()

'35° 95°'

In [85]:
daily_high_values[5].text.strip()

'39° 102°'

In [86]:
daily_high_values[0].text.strip().split()[0]

'35°'

In [87]:
daily_high_values_list = [daily_high_values[i].text.strip().split()[0] for i in range(len(daily_high_values))]
daily_high_values_list

['35°',
 '37°',
 '38°',
 '39°',
 '39°',
 '39°',
 '39°',
 '39°',
 '39°',
 '39°',
 '40°',
 '40°',
 '41°']

In [88]:
daily_low_values_list = [daily_low_values[i].text.strip().split()[0] for i in range(len(daily_low_values))]
daily_low_values_list

['26°',
 '24°',
 '23°',
 '25°',
 '25°',
 '25°',
 '26°',
 '26°',
 '26°',
 '27°',
 '27°',
 '27°',
 '28°',
 '28°']

In [89]:
daily_summary.text

'Partly cloudy and light windsThundery showers and light windsSunny intervals and light windsSunny intervals and light windsSunny intervals and light windsSunny intervals and light windsSunny intervals and light windsSunny intervals and light windsLight cloud and light windsLight cloud and light windsSunny intervals and light windsSunny intervals and a gentle breezeSunny intervals and a gentle breezeSunny intervals and a gentle breeze'

In [90]:
daily_summary_list = re.findall('[a-zA-Z][^A-Z]*', daily_summary.text)   # split the string on uppercase
daily_summary_list

['Partly cloudy and light winds',
 'Thundery showers and light winds',
 'Sunny intervals and light winds',
 'Sunny intervals and light winds',
 'Sunny intervals and light winds',
 'Sunny intervals and light winds',
 'Sunny intervals and light winds',
 'Sunny intervals and light winds',
 'Light cloud and light winds',
 'Light cloud and light winds',
 'Sunny intervals and light winds',
 'Sunny intervals and a gentle breeze',
 'Sunny intervals and a gentle breeze',
 'Sunny intervals and a gentle breeze']

In [91]:
datelist = pd.date_range(datetime.today(), periods=len(daily_high_values)).tolist()
datelist

[Timestamp('2022-05-23 18:44:58.375784', freq='D'),
 Timestamp('2022-05-24 18:44:58.375784', freq='D'),
 Timestamp('2022-05-25 18:44:58.375784', freq='D'),
 Timestamp('2022-05-26 18:44:58.375784', freq='D'),
 Timestamp('2022-05-27 18:44:58.375784', freq='D'),
 Timestamp('2022-05-28 18:44:58.375784', freq='D'),
 Timestamp('2022-05-29 18:44:58.375784', freq='D'),
 Timestamp('2022-05-30 18:44:58.375784', freq='D'),
 Timestamp('2022-05-31 18:44:58.375784', freq='D'),
 Timestamp('2022-06-01 18:44:58.375784', freq='D'),
 Timestamp('2022-06-02 18:44:58.375784', freq='D'),
 Timestamp('2022-06-03 18:44:58.375784', freq='D'),
 Timestamp('2022-06-04 18:44:58.375784', freq='D')]

In [92]:
datelist = [datelist[i].date().strftime('%y-%m-%d') for i in range(len(datelist))]
datelist

['22-05-23',
 '22-05-24',
 '22-05-25',
 '22-05-26',
 '22-05-27',
 '22-05-28',
 '22-05-29',
 '22-05-30',
 '22-05-31',
 '22-06-01',
 '22-06-02',
 '22-06-03',
 '22-06-04']

In [93]:
zipped = zip(datelist, daily_high_values_list, daily_low_values_list, daily_summary_list)

In [94]:
df = pd.DataFrame(list(zipped), columns=['Date', 'High', 'Low', 'Summary'])

In [95]:
display(df)

,Date,High,Low,Summary
0,22-05-23,35°,26°,Partly cloudy and light winds
1,22-05-24,37°,24°,Thundery showers and light winds
2,22-05-25,38°,23°,Sunny intervals and light winds
3,22-05-26,39°,25°,Sunny intervals and light winds
4,22-05-27,39°,25°,Sunny intervals and light winds
5,22-05-28,39°,25°,Sunny intervals and light winds
6,22-05-29,39°,26°,Sunny intervals and light winds
7,22-05-30,39°,26°,Sunny intervals and light winds
8,22-05-31,39°,26°,Light cloud and light winds
9,22-06-01,39°,27°,Light cloud and light winds


In [96]:
# remove the 'degree' character
df.High = df.High.replace('\°', '', regex=True).astype(float)
df.Low = df.Low.replace('\°', '', regex=True).astype(float)

In [97]:
display(df)

,Date,High,Low,Summary
0,22-05-23,35.0,26.0,Partly cloudy and light winds
1,22-05-24,37.0,24.0,Thundery showers and light winds
2,22-05-25,38.0,23.0,Sunny intervals and light winds
3,22-05-26,39.0,25.0,Sunny intervals and light winds
4,22-05-27,39.0,25.0,Sunny intervals and light winds
5,22-05-28,39.0,25.0,Sunny intervals and light winds
6,22-05-29,39.0,26.0,Sunny intervals and light winds
7,22-05-30,39.0,26.0,Sunny intervals and light winds
8,22-05-31,39.0,26.0,Light cloud and light winds
9,22-06-01,39.0,27.0,Light cloud and light winds


In [98]:
# Extract the name of the city for which date is gathered.
location = soup.find('h1', attrs={'id':'wr-location-name-id'})
location.text.split()

['Jamshedpur', '-', 'Weather', 'warnings', 'issued']

In [99]:
# create a recording
filename_csv = location.text.split()[0]+'.csv'
df.to_csv(filename_csv, index=None)

In [100]:
filename_xlsx = location.text.split()[0]+'.xlsx'
df.to_excel(filename_xlsx)